In [76]:
import numpy as np

In [124]:
from typing import Callable
def CKY_Algorithm(sequence, get_terminate_parses: Callable, beam_search_strategy: Callable = None, enable_slide_windows = False, window_size = -1):
    if (enable_slide_windows and window_size <= 0) or sequence == None or len(sequence) == 0:
        return None
        
    T = len(sequence)
    records = []
    windows_beginning_offset = 0
    for t in range(T):
        print("t = " + str(t))
        w = get_terminate_parses(sequence[t])
        
        current_record = [[]] if len(records) == 0 else [[]] * (len(records[-1]) + 1)
        current_record[-1] = w

        # When the windows size is large, we may utilize array to store records, rather than list. The array allow us adopting 
        # sliding windows more effectively when the window size is large. However, we currently utilize list to store records.
        # To do sliding window, we need first pop the first record (oldest record inside the window), and for each of rest records,
        # we need pop its first element, as the first element of each record relate to the "word" we'd like to forget.
        if enable_slide_windows:
            if len(records) == window_size:
                records.pop(0)
                for i in range(0, len(records)):
                    records[i].pop(0)
                windows_beginning_offset += 1
                
        records.append(current_record)

        for inner_record_index in range(0, len(current_record) - 1):
            current_record_length = len(current_record) 
            current_record[current_record_length - inner_record_index - 2] = []
            cell_data = []
            for split_index in range(current_record_length - inner_record_index - 2, t - windows_beginning_offset):
                # coordination = (index, t)
                cell1_coordination = (current_record_length - inner_record_index - 2, split_index)
                cell2_coordination = (split_index + 1, t - windows_beginning_offset)
                cell_data += merge_parses_in_two_cells(records[cell1_coordination[1]][cell1_coordination[0]],\
                                                       records[cell2_coordination[1]][cell2_coordination[0]],\
                                                       cell1_coordination, cell2_coordination)
            current_record[current_record_length - inner_record_index - 2] = beam_search_strategy(cell_data) if beam_search_strategy is not None else cell_data
    return records


def get_terminate_parses(w):
    return ["(" + str(w) + ", " + str(w) + ")"]

def merge_parses_in_two_cells(cell1, cell2, cell1_id = None, cell2_id = None):
    return [[cell1_id, cell2_id]]

def select_tops(cell1, beam_size = 5):
    return cell1[:beam_size]

records = CKY_Algorithm(range(0, 10), get_terminate_parses, select_tops, False)

t = 0
t = 1
t = 2
t = 3
t = 4
t = 5
t = 6
t = 7
t = 8
t = 9


In [125]:
records[-1][0] # (t, span_start_index)

[[(0, 0), (1, 9)],
 [(0, 1), (2, 9)],
 [(0, 2), (3, 9)],
 [(0, 3), (4, 9)],
 [(0, 4), (5, 9)]]

In [126]:
records

[[['(0, 0)']],
 [[[(0, 0), (1, 1)]], ['(1, 1)']],
 [[[(0, 0), (1, 2)], [(0, 1), (2, 2)]], [[(1, 1), (2, 2)]], ['(2, 2)']],
 [[[(0, 0), (1, 3)], [(0, 1), (2, 3)], [(0, 2), (3, 3)]],
  [[(1, 1), (2, 3)], [(1, 2), (3, 3)]],
  [[(2, 2), (3, 3)]],
  ['(3, 3)']],
 [[[(0, 0), (1, 4)], [(0, 1), (2, 4)], [(0, 2), (3, 4)], [(0, 3), (4, 4)]],
  [[(1, 1), (2, 4)], [(1, 2), (3, 4)], [(1, 3), (4, 4)]],
  [[(2, 2), (3, 4)], [(2, 3), (4, 4)]],
  [[(3, 3), (4, 4)]],
  ['(4, 4)']],
 [[[(0, 0), (1, 5)],
   [(0, 1), (2, 5)],
   [(0, 2), (3, 5)],
   [(0, 3), (4, 5)],
   [(0, 4), (5, 5)]],
  [[(1, 1), (2, 5)], [(1, 2), (3, 5)], [(1, 3), (4, 5)], [(1, 4), (5, 5)]],
  [[(2, 2), (3, 5)], [(2, 3), (4, 5)], [(2, 4), (5, 5)]],
  [[(3, 3), (4, 5)], [(3, 4), (5, 5)]],
  [[(4, 4), (5, 5)]],
  ['(5, 5)']],
 [[[(0, 0), (1, 6)],
   [(0, 1), (2, 6)],
   [(0, 2), (3, 6)],
   [(0, 3), (4, 6)],
   [(0, 4), (5, 6)]],
  [[(1, 1), (2, 6)],
   [(1, 2), (3, 6)],
   [(1, 3), (4, 6)],
   [(1, 4), (5, 6)],
   [(1, 5), (6, 6)]],
  